In [2]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    classification_report, confusion_matrix,
    f1_score, accuracy_score, precision_score, recall_score,
    roc_auc_score, roc_curve
)
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTETomek
import warnings
warnings.filterwarnings('ignore')


class AvaliacaoCompleta:
    """Classe para avaliação completa do modelo com análise de desbalanceamento"""

    def __init__(self, filepath, target_column=None, features_selecionadas=None):
        """
        Inicializa a avaliação

        Args:
            filepath: Caminho para o arquivo de dados
            target_column: Nome da coluna target
            features_selecionadas: Lista de features selecionadas
        """
        print("=" * 100)
        print(" " * 30 + "AVALIAÇÃO COMPLETA DO MODELO")
        print("=" * 100 + "\n")

        # Carregar dados
        if filepath.endswith('.csv'):
            self.df = pd.read_csv(filepath)
        else:
            self.df = pd.read_excel(filepath)

        # Definir target
        if target_column is None:
            target_column = self.df.columns[-1]

        self.target_column = target_column

        # Selecionar features
        if features_selecionadas is None:
            self.X = self.df.drop(columns=[target_column])
        else:
            self.X = self.df[features_selecionadas]

        self.y = self.df[target_column]

        print(f"Dataset carregado: {self.df.shape[0]} exemplos, {self.X.shape[1]} features")
        print(f"Target: {target_column}")

        # Analisar desbalanceamento
        self._analisar_desbalanceamento()

    def _analisar_desbalanceamento(self):
        """Analisa o desbalanceamento das classes"""
        print("\n" + "=" * 100)
        print("ANÁLISE DE DESBALANCEAMENTO")
        print("=" * 100)

        contagem = self.y.value_counts().sort_index()
        proporcoes = self.y.value_counts(normalize=True).sort_index() * 100

        print("\nDistribuição Original das Classes:\n")
        print("┌─────────────┬────────────┬─────────────┐")
        print("│   Classe    │  Exemplos  │ Proporção % │")
        print("├─────────────┼────────────┼─────────────┤")
        for classe in contagem.index:
            count = contagem[classe]
            prop = proporcoes[classe]
            print(f"│   Classe {classe}  │   {count:5d}    │    {prop:5.2f}%   │")
        print("└─────────────┴────────────┴─────────────┘")

        # Calcular razão
        razao = contagem.max() / contagem.min()
        self.razao_desbalanceamento = razao

        print(f"\nRazão de Desbalanceamento: {razao:.2f}:1")

        if razao > 2:
            nivel = "SEVERO"
        elif razao > 1.5:
            nivel = "MODERADO"
        else:
            nivel = "BAIXO"


        print(f"Nível de Desbalanceamento: {nivel}")

        if razao > 1.5:
            print("\n IMPACTO ESPERADO DO DESBALANCEAMENTO:")
            print("   • Modelo pode ter viés para a classe majoritária")
            print("   • Métricas como Acurácia podem ser enganosas")
            print("   • Classe minoritária pode ter baixo Recall")
            print("   • Técnicas de balanceamento são NECESSÁRIAS")

    def comparar_tecnicas_balanceamento(self, modelo_nome='Random Forest',
                                       features_selecionadas=None, cv_folds=5):
        """
        Compara o desempenho do modelo SEM e COM diferentes técnicas de balanceamento

        Args:
            modelo_nome: Nome do modelo a usar
            features_selecionadas: Lista de features
            cv_folds: Número de folds
        """
        print("\n" + "=" * 100)
        print("COMPARAÇÃO: SEM vs COM BALANCEAMENTO")
        print("=" * 100)

        # Preparar dados
        if features_selecionadas is None:
            X = self.X
        else:
            X = self.X[features_selecionadas]

        # Split
        X_train, X_test, y_train, y_test = train_test_split(
            X, self.y, test_size=0.2, random_state=42, stratify=self.y
        )

        # Padronizar
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        # Selecionar modelo
        if modelo_nome == 'Random Forest':
            modelo_base = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)
        elif modelo_nome == 'Gradient Boosting':
            modelo_base = GradientBoostingClassifier(n_estimators=200, random_state=42)
        else:
            modelo_base = LogisticRegression(max_iter=1000, random_state=42)

        resultados = {}

        # 1. SEM BALANCEAMENTO (BASELINE)
        print("\nTestando SEM balanceamento (Baseline)...")
        modelo = modelo_base.__class__(**modelo_base.get_params())
        modelo.fit(X_train_scaled, y_train)
        y_pred = modelo.predict(X_test_scaled)

        resultados['SEM Balanceamento'] = self._calcular_metricas(y_test, y_pred, "Baseline")

        # 2. COM SMOTE
        print("\nTestando COM SMOTE...")
        smote = SMOTE(random_state=42)
        X_smote, y_smote = smote.fit_resample(X_train_scaled, y_train)

        modelo = modelo_base.__class__(**modelo_base.get_params())
        modelo.fit(X_smote, y_smote)
        y_pred = modelo.predict(X_test_scaled)

        resultados['SMOTE'] = self._calcular_metricas(y_test, y_pred, "SMOTE")

        # 3. COM ADASYN
        print("\nTestando COM ADASYN...")
        try:
            adasyn = ADASYN(random_state=42)
            X_adasyn, y_adasyn = adasyn.fit_resample(X_train_scaled, y_train)

            modelo = modelo_base.__class__(**modelo_base.get_params())
            modelo.fit(X_adasyn, y_adasyn)
            y_pred = modelo.predict(X_test_scaled)

            resultados['ADASYN'] = self._calcular_metricas(y_test, y_pred, "ADASYN")
        except:
            print("ADASYN falhou (possível falta de vizinhos)")
            resultados['ADASYN'] = None

        # 4. COM SMOTE + Tomek
        print("\nTestando COM SMOTE + Tomek Links...")
        smotetomek = SMOTETomek(random_state=42)
        X_st, y_st = smotetomek.fit_resample(X_train_scaled, y_train)

        modelo = modelo_base.__class__(**modelo_base.get_params())
        modelo.fit(X_st, y_st)
        y_pred = modelo.predict(X_test_scaled)

        resultados['SMOTE+Tomek'] = self._calcular_metricas(y_test, y_pred, "SMOTE+Tomek")

        # 5. COM Under-sampling
        print("\nTestando COM Under-Sampling...")
        under = RandomUnderSampler(random_state=42)
        X_under, y_under = under.fit_resample(X_train_scaled, y_train)

        modelo = modelo_base.__class__(**modelo_base.get_params())
        modelo.fit(X_under, y_under)
        y_pred = modelo.predict(X_test_scaled)

        resultados['Under-Sampling'] = self._calcular_metricas(y_test, y_pred, "Under-Sampling")

        self.resultados_comparacao = resultados
        self.X_test = X_test_scaled
        self.y_test = y_test

        return resultados

    def _calcular_metricas(self, y_true, y_pred, nome_tecnica):
        """Calcula todas as métricas necessárias"""

        metricas = {
            'Acurácia': accuracy_score(y_true, y_pred),
            'Macro F1-Score': f1_score(y_true, y_pred, average='macro'),
            'Weighted F1-Score': f1_score(y_true, y_pred, average='weighted'),
            'Macro Precision': precision_score(y_true, y_pred, average='macro', zero_division=0),
            'Macro Recall': recall_score(y_true, y_pred, average='macro'),
            'Confusion Matrix': confusion_matrix(y_true, y_pred)
        }

        # Métricas por classe
        report = classification_report(y_true, y_pred, output_dict=True)
        metricas['Report'] = report

        print(f"   ✓ {nome_tecnica} - Macro F1: {metricas['Macro F1-Score']:.4f}, Acurácia: {metricas['Acurácia']:.4f}")

        return metricas

    def gerar_tabela_comparativa(self):
        """Gera tabela comparativa das técnicas"""
        print("\n" + "=" * 100)
        print("TABELA COMPARATIVA DE TÉCNICAS")
        print("=" * 100 + "\n")

        # Criar DataFrame
        data = []
        for tecnica, metricas in self.resultados_comparacao.items():
            if metricas is not None:
                data.append({
                    'Técnica': tecnica,
                    'Acurácia': metricas['Acurácia'],
                    'Macro F1-Score': metricas['Macro F1-Score'],
                    'Weighted F1-Score': metricas['Weighted F1-Score'],
                    'Macro Precision': metricas['Macro Precision'],
                    'Macro Recall': metricas['Macro Recall']
                })

        df_comp = pd.DataFrame(data)
        df_comp = df_comp.round(4)

        print(df_comp.to_string(index=False))

        # Identificar melhor técnica
        melhor_idx = df_comp['Macro F1-Score'].idxmax()
        melhor_tecnica = df_comp.iloc[melhor_idx]['Técnica']
        melhor_f1 = df_comp.iloc[melhor_idx]['Macro F1-Score']

        baseline_f1 = df_comp[df_comp['Técnica'] == 'SEM Balanceamento']['Macro F1-Score'].values[0]
        melhoria = ((melhor_f1 - baseline_f1) / baseline_f1) * 100

        print("\n" + "─" * 100)
        print(f"MELHOR TÉCNICA: {melhor_tecnica}")
        print(f"   Macro F1-Score: {melhor_f1:.4f}")
        print(f"   Melhoria sobre Baseline: +{melhoria:.2f}%")
        print("─" * 100)

        self.df_comparacao = df_comp
        self.melhor_tecnica = melhor_tecnica

        return df_comp

    def visualizar_comparacao(self, output_file='comparacao_tecnicas.png'):
        """Cria visualizações comparativas"""
        print(f"\n Gerando visualizações...")

        fig, axes = plt.subplots(2, 3, figsize=(20, 12))
        fig.suptitle('Comparação de Técnicas de Balanceamento',
                     fontsize=18, fontweight='bold', y=0.995)

        # 1. Acurácia
        ax = axes[0, 0]
        self.df_comparacao.plot(x='Técnica', y='Acurácia', kind='bar',
                                ax=ax, color='steelblue', legend=False)
        ax.set_title('Acurácia', fontsize=14, fontweight='bold')
        ax.set_ylabel('Score', fontsize=12)
        ax.set_xlabel('')
        ax.set_ylim([0.85, 1.0])
        ax.grid(axis='y', alpha=0.3)
        ax.tick_params(axis='x', rotation=45)

        # Destacar melhor
        melhor_idx = self.df_comparacao['Acurácia'].idxmax()
        ax.patches[melhor_idx].set_color('gold')
        ax.patches[melhor_idx].set_edgecolor('black')
        ax.patches[melhor_idx].set_linewidth(3)

        # 2. Macro F1-Score
        ax = axes[0, 1]
        self.df_comparacao.plot(x='Técnica', y='Macro F1-Score', kind='bar',
                                ax=ax, color='coral', legend=False)
        ax.set_title('Macro F1-Score (Métrica Principal)', fontsize=14, fontweight='bold')
        ax.set_ylabel('Score', fontsize=12)
        ax.set_xlabel('')
        ax.set_ylim([0.85, 1.0])
        ax.grid(axis='y', alpha=0.3)
        ax.tick_params(axis='x', rotation=45)

        # Destacar melhor
        melhor_idx = self.df_comparacao['Macro F1-Score'].idxmax()
        ax.patches[melhor_idx].set_color('gold')
        ax.patches[melhor_idx].set_edgecolor('black')
        ax.patches[melhor_idx].set_linewidth(3)

        # 3. Weighted F1-Score
        ax = axes[0, 2]
        self.df_comparacao.plot(x='Técnica', y='Weighted F1-Score', kind='bar',
                                ax=ax, color='mediumseagreen', legend=False)
        ax.set_title('Weighted F1-Score', fontsize=14, fontweight='bold')
        ax.set_ylabel('Score', fontsize=12)
        ax.set_xlabel('')
        ax.set_ylim([0.85, 1.0])
        ax.grid(axis='y', alpha=0.3)
        ax.tick_params(axis='x', rotation=45)

        # 4. Macro Precision
        ax = axes[1, 0]
        self.df_comparacao.plot(x='Técnica', y='Macro Precision', kind='bar',
                                ax=ax, color='plum', legend=False)
        ax.set_title('Macro Precision', fontsize=14, fontweight='bold')
        ax.set_ylabel('Score', fontsize=12)
        ax.set_xlabel('')
        ax.set_ylim([0.85, 1.0])
        ax.grid(axis='y', alpha=0.3)
        ax.tick_params(axis='x', rotation=45)

        # 5. Macro Recall
        ax = axes[1, 1]
        self.df_comparacao.plot(x='Técnica', y='Macro Recall', kind='bar',
                                ax=ax, color='lightsalmon', legend=False)
        ax.set_title('Macro Recall', fontsize=14, fontweight='bold')
        ax.set_ylabel('Score', fontsize=12)
        ax.set_xlabel('')
        ax.set_ylim([0.85, 1.0])
        ax.grid(axis='y', alpha=0.3)
        ax.tick_params(axis='x', rotation=45)

        # 6. Comparação lado a lado
        ax = axes[1, 2]
        metricas_principais = self.df_comparacao[['Técnica', 'Acurácia', 'Macro F1-Score',
                                                   'Macro Precision', 'Macro Recall']].set_index('Técnica')
        metricas_principais.plot(kind='bar', ax=ax, width=0.8)
        ax.set_title('Todas as Métricas', fontsize=14, fontweight='bold')
        ax.set_ylabel('Score', fontsize=12)
        ax.set_xlabel('')
        ax.set_ylim([0.85, 1.0])
        ax.legend(loc='lower right', fontsize=9)
        ax.grid(axis='y', alpha=0.3)
        ax.tick_params(axis='x', rotation=45)

        plt.tight_layout()
        plt.savefig(output_file, dpi=300, bbox_inches='tight')
        print(f"   ✓ Gráficos salvos em: {output_file}")
        plt.close()

    def visualizar_matrizes_confusao(self, output_file='matrizes_confusao.png'):
        """Cria visualização das matrizes de confusão"""
        print(f"\nGerando matrizes de confusão...")

        # Filtrar técnicas válidas
        tecnicas_validas = {k: v for k, v in self.resultados_comparacao.items() if v is not None}
        n_tecnicas = len(tecnicas_validas)

        # Criar subplots
        fig, axes = plt.subplots(2, 3, figsize=(18, 12))
        fig.suptitle('Matrizes de Confusão - Comparação de Técnicas',
                     fontsize=16, fontweight='bold')
        axes = axes.flatten()

        for idx, (tecnica, metricas) in enumerate(tecnicas_validas.items()):
            cm = metricas['Confusion Matrix']

            # Calcular percentuais
            cm_percentual = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100

            # Plotar
            sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[idx],
                       cbar=False, square=True, linewidths=2, linecolor='white',
                       annot_kws={'size': 14, 'weight': 'bold'})

            # Adicionar percentuais
            for i in range(cm.shape[0]):
                for j in range(cm.shape[1]):
                    axes[idx].text(j + 0.5, i + 0.7, f'({cm_percentual[i, j]:.1f}%)',
                                 ha='center', va='center', fontsize=10, color='gray')

            axes[idx].set_title(f'{tecnica}\nF1-Macro: {metricas["Macro F1-Score"]:.4f}',
                              fontsize=12, fontweight='bold')
            axes[idx].set_ylabel('Classe Real', fontsize=11)
            axes[idx].set_xlabel('Classe Predita', fontsize=11)

            # Destacar melhor técnica
            if tecnica == self.melhor_tecnica:
                for spine in axes[idx].spines.values():
                    spine.set_edgecolor('gold')
                    spine.set_linewidth(4)

        # Remover subplots não utilizados
        for idx in range(n_tecnicas, len(axes)):
            fig.delaxes(axes[idx])

        plt.tight_layout()
        plt.savefig(output_file, dpi=300, bbox_inches='tight')
        print(f"   ✓ Matrizes de confusão salvas em: {output_file}")
        plt.close()

    def analisar_metricas_por_classe(self):
        """Analisa métricas detalhadas por classe"""
        print("\n" + "=" * 100)
        print("ANÁLISE DETALHADA POR CLASSE")
        print("=" * 100)

        # Comparar baseline vs melhor técnica
        baseline = self.resultados_comparacao['SEM Balanceamento']
        melhor = self.resultados_comparacao[self.melhor_tecnica]

        print(f"\nComparação: BASELINE vs {self.melhor_tecnica}\n")

        # Criar tabela
        classes = sorted(baseline['Report'].keys())
        classes = [c for c in classes if c not in ['accuracy', 'macro avg', 'weighted avg']]

        print("┌─────────┬──────────────────────────────┬──────────────────────────────┐")
        print(f"│ Classe  │      SEM Balanceamento       │       {self.melhor_tecnica:^20s}       │")
        print("├─────────┼──────────────────────────────┼──────────────────────────────┤")
        print("│         │  Prec   Recall   F1-Score    │  Prec   Recall   F1-Score    │")
        print("├─────────┼──────────────────────────────┼──────────────────────────────┤")

        for classe in classes:
            base_metrics = baseline['Report'][str(classe)]
            melhor_metrics = melhor['Report'][str(classe)]

            print(f"│ Classe {classe} │  {base_metrics['precision']:.3f}   {base_metrics['recall']:.3f}    {base_metrics['f1-score']:.3f}     │" +
                  f"  {melhor_metrics['precision']:.3f}   {melhor_metrics['recall']:.3f}    {melhor_metrics['f1-score']:.3f}     │")

        print("└─────────┴──────────────────────────────┴──────────────────────────────┘")

        # Análise de melhorias
        print("\nMELHORIAS POR CLASSE:\n")
        for classe in classes:
            base_f1 = baseline['Report'][str(classe)]['f1-score']
            melhor_f1 = melhor['Report'][str(classe)]['f1-score']
            melhoria = ((melhor_f1 - base_f1) / base_f1) * 100

            seta = "^" if melhoria > 0 else "" if melhoria < 0 else ">"
            print(f"   Classe {classe}: {seta} {melhoria:+.2f}% (de {base_f1:.4f} para {melhor_f1:.4f})")

    def gerar_relatorio_final(self, output_file='RELATORIO_AVALIACAO.txt'):
        """Gera relatório final completo"""
        print(f"\nGerando relatório final...")

        with open(output_file, 'w', encoding='utf-8') as f:
            f.write("=" * 100 + "\n")
            f.write(" " * 25 + "RELATÓRIO DE AVALIAÇÃO DO MODELO\n")
            f.write(" " * 20 + "Análise do Impacto do Desbalanceamento\n")
            f.write("=" * 100 + "\n\n")

            # 1. INFORMAÇÕES DO DATASET
            f.write("1. INFORMAÇÕES DO DATASET\n")
            f.write("-" * 100 + "\n\n")
            f.write(f"Total de Exemplos: {len(self.df)}\n")
            f.write(f"Número de Features: {self.X.shape[1]}\n")
            f.write(f"Target: {self.target_column}\n\n")

            # Distribuição
            contagem = self.y.value_counts().sort_index()
            proporcoes = self.y.value_counts(normalize=True).sort_index() * 100

            f.write("Distribuição de Classes:\n")
            for classe in contagem.index:
                f.write(f"  Classe {classe}: {contagem[classe]:5d} exemplos ({proporcoes[classe]:5.2f}%)\n")

            f.write(f"\nRazão de Desbalanceamento: {self.razao_desbalanceamento:.2f}:1\n")

            # 2. IMPACTO DO DESBALANCEAMENTO
            f.write("\n\n2. IMPACTO DO DESBALANCEAMENTO\n")
            f.write("-" * 100 + "\n\n")

            baseline = self.resultados_comparacao['SEM Balanceamento']

            f.write("MODELO SEM BALANCEAMENTO (Baseline):\n\n")
            f.write(f"  • Acurácia:        {baseline['Acurácia']:.4f}\n")
            f.write(f"  • Macro F1-Score:  {baseline['Macro F1-Score']:.4f}\n")
            f.write(f"  • Macro Precision: {baseline['Macro Precision']:.4f}\n")
            f.write(f"  • Macro Recall:    {baseline['Macro Recall']:.4f}\n\n")

            f.write("PROBLEMAS IDENTIFICADOS:\n\n")

            # Analisar problemas
            report = baseline['Report']
            classes = [c for c in report.keys() if c not in ['accuracy', 'macro avg', 'weighted avg']]

            # Identificar classe minoritária
            contagem = self.y.value_counts()
            classe_minoritaria = str(contagem.idxmin())
            classe_majoritaria = str(contagem.idxmax())

            recall_min = report[classe_minoritaria]['recall']
            recall_maj = report[classe_majoritaria]['recall']

            if recall_min < recall_maj - 0.05:
                f.write(f"   Classe minoritária ({classe_minoritaria}) com Recall significativamente menor\n")
                f.write(f"      Recall Classe {classe_minoritaria}: {recall_min:.4f}\n")
                f.write(f"      Recall Classe {classe_majoritaria}: {recall_maj:.4f}\n")
                f.write(f"      Diferença: {(recall_maj - recall_min):.4f}\n\n")

            f.write("  Modelo pode estar enviesado para a classe majoritária\n")
            f.write("  Acurácia alta pode ser enganosa devido ao desbalanceamento\n")
            f.write("  Macro F1-Score é mais confiável que Acurácia neste caso\n")

            # 3. TÉCNICAS APLICADAS
            f.write("\n\n3. TÉCNICAS DE BALANCEAMENTO APLICADAS\n")
            f.write("-" * 100 + "\n\n")

            f.write("Foram testadas 5 abordagens:\n\n")

            f.write("1. SEM Balanceamento (Baseline)\n")
            f.write("   • Modelo treinado com dados originais desbalanceados\n")
            f.write("   • Serve como referência para medir melhorias\n\n")

            f.write("2. SMOTE (Synthetic Minority Over-sampling Technique)\n")
            f.write("   • Cria exemplos sintéticos da classe minoritária\n")
            f.write("   • Gera novos pontos interpolando exemplos existentes\n")
            f.write("   • Aumenta o tamanho do conjunto de treino\n\n")

            f.write("3. ADASYN (Adaptive Synthetic Sampling)\n")
            f.write("   • Similar ao SMOTE, mas adaptativo\n")
            f.write("   • Foca em gerar exemplos em regiões difíceis de classificar\n")
            f.write("   • Mais sensível a outliers\n\n")

            f.write("4. SMOTE + Tomek Links\n")
            f.write("   • Combina SMOTE com limpeza de fronteira\n")
            f.write("   • Remove exemplos ambíguos após sobre-amostragem\n")
            f.write("   • Melhora a definição da fronteira de decisão\n\n")

            f.write("5. Under-Sampling Aleatório\n")
            f.write("   • Remove aleatoriamente exemplos da classe majoritária\n")
            f.write("   • Reduz o tamanho do conjunto de treino\n")
            f.write("   • Pode perder informação importante\n\n")

            # 4. RESULTADOS COMPARATIVOS
            f.write("\n4. RESULTADOS COMPARATIVOS\n")
            f.write("-" * 100 + "\n\n")

            f.write(self.df_comparacao.to_string(index=False))
            f.write("\n\n")

            # 5. MELHOR TÉCNICA
            f.write("\n5. MELHOR TÉCNICA IDENTIFICADA\n")
            f.write("-" * 100 + "\n\n")

            melhor = self.resultados_comparacao[self.melhor_tecnica]
            baseline_f1 = baseline['Macro F1-Score']
            melhor_f1 = melhor['Macro F1-Score']
            melhoria = ((melhor_f1 - baseline_f1) / baseline_f1) * 100

            f.write(f"TÉCNICA VENCEDORA: {self.melhor_tecnica}\n\n")
            f.write(f"MÉTRICAS FINAIS:\n\n")
            f.write(f"  • Acurácia:        {melhor['Acurácia']:.4f}\n")
            f.write(f"  • Macro F1-Score:  {melhor['Macro F1-Score']:.4f}\n")
            f.write(f"  • Macro Precision: {melhor['Macro Precision']:.4f}\n")
            f.write(f"  • Macro Recall:    {melhor['Macro Recall']:.4f}\n\n")

            f.write(f"MELHORIA SOBRE BASELINE:\n\n")
            f.write(f"  • Macro F1-Score:  +{melhoria:.2f}%\n")
            f.write(f"  • De {baseline_f1:.4f} para {melhor_f1:.4f}\n\n")

            # 6. MATRIZ DE CONFUSÃO
            f.write("\n6. MATRIZ DE CONFUSÃO - MELHOR TÉCNICA\n")
            f.write("-" * 100 + "\n\n")

            cm = melhor['Confusion Matrix']
            f.write("Matriz de Confusão:\n\n")
            f.write(str(cm))
            f.write("\n\n")

            # Análise da matriz
            f.write("INTERPRETAÇÃO:\n\n")

            # Para classificação binária
            if cm.shape[0] == 2:
                tn, fp, fn, tp = cm.ravel()

                f.write(f"  Verdadeiros Negativos (TN): {tn}\n")
                f.write(f"  Falsos Positivos (FP):      {fp}\n")
                f.write(f"  Falsos Negativos (FN):      {fn}\n")
                f.write(f"  Verdadeiros Positivos (TP): {tp}\n\n")

                # Calcular taxas
                tpr = tp / (tp + fn) if (tp + fn) > 0 else 0
                tnr = tn / (tn + fp) if (tn + fp) > 0 else 0

                f.write(f"  Taxa de Verdadeiros Positivos (Recall Classe 1): {tpr:.4f}\n")
                f.write(f"  Taxa de Verdadeiros Negativos (Recall Classe 0): {tnr:.4f}\n\n")

                if abs(tpr - tnr) < 0.05:
                    f.write("  Modelo balanceado entre as classes\n")
                else:
                    f.write(" Modelo ainda apresenta algum viés\n")

            # 7. ANÁLISE POR CLASSE
            f.write("\n\n7. DESEMPENHO POR CLASSE\n")
            f.write("-" * 100 + "\n\n")

            f.write("COMPARAÇÃO: Baseline vs Melhor Técnica\n\n")

            classes = [c for c in melhor['Report'].keys() if c not in ['accuracy', 'macro avg', 'weighted avg']]

            for classe in classes:
                base_metrics = baseline['Report'][str(classe)]
                melhor_metrics = melhor['Report'][str(classe)]

                f.write(f"Classe {classe}:\n")
                f.write(f"  Baseline:      Precision={base_metrics['precision']:.4f}, " +
                       f"Recall={base_metrics['recall']:.4f}, F1={base_metrics['f1-score']:.4f}\n")
                f.write(f"  {self.melhor_tecnica}: Precision={melhor_metrics['precision']:.4f}, " +
                       f"Recall={melhor_metrics['recall']:.4f}, F1={melhor_metrics['f1-score']:.4f}\n")

                melhoria_f1 = ((melhor_metrics['f1-score'] - base_metrics['f1-score']) /
                              base_metrics['f1-score']) * 100
                f.write(f"  Melhoria F1:   {melhoria_f1:+.2f}%\n\n")

            # 8. CONCLUSÕES
            f.write("\n8. CONCLUSÕES E RECOMENDAÇÕES\n")
            f.write("-" * 100 + "\n\n")

            f.write("PRINCIPAIS DESCOBERTAS:\n\n")

            f.write(f"1. O dataset apresenta desbalanceamento {self.razao_desbalanceamento:.2f}:1\n\n")

            f.write(f"2. O uso de {self.melhor_tecnica} melhorou significativamente o desempenho:\n")
            f.write(f"   • Macro F1-Score aumentou em {melhoria:.2f}%\n")
            f.write(f"   • Modelo ficou mais balanceado entre as classes\n\n")

            f.write("3. Métricas adequadas para dados desbalanceados:\n")
            f.write("   • Macro F1-Score é a métrica principal (equilibra precisão e recall)\n")
            f.write("   • Matriz de Confusão revela padrões de erro por classe\n")
            f.write("   • Acurácia sozinha pode ser enganosa\n\n")

            f.write("RECOMENDAÇÕES:\n\n")
            f.write(f"• Utilizar {self.melhor_tecnica} para treinar o modelo final\n")
            f.write("• Monitorar Macro F1-Score como métrica principal\n")
            f.write("• Analisar a Matriz de Confusão regularmente\n")
            f.write("• Considerar ajuste de hiperparâmetros para melhorias adicionais\n")

            f.write("\n" + "=" * 100 + "\n")
            f.write("RELATÓRIO GERADO COM SUCESSO\n")
            f.write("=" * 100 + "\n")

        print(f"   ✓ Relatório salvo em: {output_file}")



if __name__ == "__main__":

    FILEPATH = 'datasample.csv'
    TARGET_COLUMN = 'Class'

    FEATURES_SELECIONADAS = [
        'V14', 'V12', 'V4', 'V11', 'V10',
        'V16', 'V17', 'V3', 'V9', 'V7'
    ]

    MODELO = 'Random Forest'

    try:
        # 1. Inicializar avaliação
        avaliacao = AvaliacaoCompleta(
            filepath=FILEPATH,
            target_column=TARGET_COLUMN,
            features_selecionadas=FEATURES_SELECIONADAS
        )

        # 2. Comparar técnicas de balanceamento
        resultados = avaliacao.comparar_tecnicas_balanceamento(
            modelo_nome=MODELO,
            features_selecionadas=FEATURES_SELECIONADAS
        )

        # 3. Gerar tabela comparativa
        df_comparacao = avaliacao.gerar_tabela_comparativa()

        # 4. Visualizações
        avaliacao.visualizar_comparacao('comparacao_tecnicas.png')
        avaliacao.visualizar_matrizes_confusao('matrizes_confusao.png')

        # 5. Análise por classe
        avaliacao.analisar_metricas_por_classe()

        # 6. Relatório final
        avaliacao.gerar_relatorio_final('RELATORIO_AVALIACAO.txt')

        print("\n" + "=" * 100)
        print("AVALIAÇÃO COMPLETA CONCLUÍDA COM SUCESSO!")
        print("=" * 100)
        print("\nArquivos gerados:")
        print("   • RELATORIO_AVALIACAO.txt - Relatório completo")
        print("   • comparacao_tecnicas.png - Gráficos comparativos")
        print("   • matrizes_confusao.png   - Matrizes de confusão")

    except Exception as e:
        print(f"\nERRO: {str(e)}")
        import traceback
        traceback.print_exc()

                              AVALIAÇÃO COMPLETA DO MODELO

Dataset carregado: 1759 exemplos, 10 features
Target: Class

ANÁLISE DE DESBALANCEAMENTO

Distribuição Original das Classes:

┌─────────────┬────────────┬─────────────┐
│   Classe    │  Exemplos  │ Proporção % │
├─────────────┼────────────┼─────────────┤
│   Classe 0  │    1267    │    72.03%   │
│   Classe 1  │     492    │    27.97%   │
└─────────────┴────────────┴─────────────┘

Razão de Desbalanceamento: 2.58:1
Nível de Desbalanceamento: SEVERO

 IMPACTO ESPERADO DO DESBALANCEAMENTO:
   • Modelo pode ter viés para a classe majoritária
   • Métricas como Acurácia podem ser enganosas
   • Classe minoritária pode ter baixo Recall
   • Técnicas de balanceamento são NECESSÁRIAS

COMPARAÇÃO: SEM vs COM BALANCEAMENTO

Testando SEM balanceamento (Baseline)...
   ✓ Baseline - Macro F1: 0.9372, Acurácia: 0.9517

Testando COM SMOTE...
   ✓ SMOTE - Macro F1: 0.9308, Acurácia: 0.9460

Testando COM ADASYN...
   ✓ ADASYN - Macro F1: 0.91